# Rauda AI - Evaluación automática de tickets

Este notebook ejecuta la prueba técnica de forma guiada:

1) Comprueba que el entorno esté preparado (`python3.12 -m venv .venv` + `pip install -r requirements.txt`).
2) Ejecuta la evaluación con OpenAI y guarda `tickets_evaluated.csv`.
3) Verifica que el CSV de salida cumple con la especificación.


## Paso 0. Requisitos previos

Antes de ejecutar:

1) Exporta tu OpenAI API key en la terminal donde ejecutas el notebook: `export OPENAI_API_KEY=tu_clave`

2) Activa tu `.venv` e inicia Jupyter/VS Code usando ese entorno para ejecutar este notebook.

3) Si no ves el kernel correcto, abre la paleta de comandos y ejecuta `Developer: Reload Window`.


In [1]:
from pathlib import Path
import pandas as pd
from ticket_evaluator.evaluate_tickets import evaluate_tickets


## Paso 1. Ejecutar evaluación
El resultado se guarda en `tickets_evaluated.csv` y reutiliza una sola función (`evaluate_tickets`).
La parte de LLM usa `responses.create` con `text.format` y `json_schema` en modo `strict`, y valida la salida contra `TicketEvaluation`.

In [2]:
INPUT_CSV = 'tickets.csv'
OUTPUT_CSV = 'tickets_evaluated.csv'
if not Path(INPUT_CSV).exists():
    raise FileNotFoundError(f'No se encontró {INPUT_CSV}. Coloca el CSV en la raíz del repositorio.')

output_path = None
try:
    output_path = evaluate_tickets(
        input_csv=INPUT_CSV,
        output_csv=OUTPUT_CSV,
        model='gpt-4o',
        max_rows=0,
        request_timeout=60,
        max_retries=3,
        skip_api=False,
    )
    print(f'Archivo generado: {output_path}')
except RuntimeError as exc:
    print('❌ No se pudo ejecutar el paso de evaluación.')
    print(f'   {exc}')
    print('Revisa que OPENAI_API_KEY esté exportada en tu terminal y vuelve a correr solo esta celda.')


Saved 5 evaluated rows to: tickets_evaluated.csv
Archivo generado: tickets_evaluated.csv


## Paso 2. Prueba de validación

Verifica que el archivo de salida tenga las columnas esperadas, el mismo número de filas y notas de texto no vacías.

In [3]:
if output_path is None:
    raise RuntimeError('No se generó tickets_evaluated.csv. Corrige la API key y vuelve a ejecutar el Paso 1.')

output_csv = Path(OUTPUT_CSV)
required_cols = {
    'ticket', 'reply',
    'content_score', 'content_explanation',
    'format_score', 'format_explanation',
}
errors = []

if not output_csv.exists():
    raise FileNotFoundError(f'No se encontró {output_csv}')

df_in = pd.read_csv(INPUT_CSV)
df_out = pd.read_csv(output_csv)

if set(required_cols) - set(df_out.columns):
    errors.append(f'Faltan columnas: {sorted(required_cols - set(df_out.columns))}')
if len(df_out) != len(df_in):
    errors.append(f'Número de filas distinto: entrada={len(df_in)} salida={len(df_out)}')

for col in ['content_score', 'format_score']:
    numeric = pd.to_numeric(df_out[col], errors='coerce')
    invalid = df_out[
        numeric.isna()
        | (numeric < 1)
        | (numeric > 5)
        | (numeric != numeric.round())
    ]
    if not invalid.empty:
        errors.append(f'Valores inválidos en {col}. Deben ser enteros 1-5.')

for col in ['content_explanation', 'format_explanation']:
    empty = df_out[col].astype(str).str.strip().eq('')
    if empty.any():
        errors.append(f'Hay explicaciones vacías en {col}.')

if errors:
    print('❌ El resultado NO cumple la especificación:')
    for issue in errors:
        print('-', issue)
else:
    print(f'✅ OK: se validan {len(df_out)} filas y todas cumplen el formato esperado.')
    pd.set_option('display.max_rows', None)
    pd.set_option('display.max_columns', None)
    display(df_out)


✅ OK: se validan 5 filas y todas cumplen el formato esperado.


,ticket,reply,content_score,content_explanation,format_score,format_explanation
0,"Hi, I'd like to check the shipping status of m...","Sure, you can check your shipping status on ou...",5,The reply provides the requested information a...,5,"The reply is clear, concise, and grammatically..."
1,The product I received is defective. I'd like ...,We're sorry to hear that. Could you please pro...,3,The reply asks for a photo but does not provid...,5,"The reply is clear, concise, and professionall..."
2,I have a question about how to set up my new d...,"Hello, thanks for contacting us. Please go to ...",3,The reply provides a general direction but lac...,5,"The reply is clear, concise, and professionall..."
3,"Hello, my account was charged twice. Could you...",We've identified the issue and have requested ...,5,The reply directly addresses the issue by conf...,5,"The reply is clear, concise, and professionall..."
4,I can't log into my account. It says 'invalid ...,Please reset your password using the 'Forgot P...,5,The reply provides a direct and actionable sol...,5,"The reply is clear, concise, and professionall..."
